In [1]:
import rasterio

In [ ]:
input_sar_image_path  = "../SAR_Data_processing_eaton/8_subset/eaton_S1A_IW_GRDH_1SDV_stack_32611_clip_bilinear_10m.tif"
glcm_raster_path="../SAR_Data_processing_eaton/9_glcm/eaton_S1A_IW_GRDH_1SDV_Cal_Spk_TC_stack_glcm_32611_bilinear_10.tif"
thermal_image_path="../ThermalData/05_resample/TRAD_diff_norm_bilinear_10.tif"
dnbr_image_path="../S2_Data_Processing/Indices/dNBR_eaton_bilinear_10m.tif"

images_dict={"input_sar_image_path":input_sar_image_path,
             "glcm_raster_path":glcm_raster_path,
             "thermal_image_path":thermal_image_path,
             "dnbr_image_path":dnbr_image_path,
             }

# images_dict={"input_sar_image_path":input_sar_image_path,
#              "glcm_raster_path":"../SAR-Burnt-Area-Mapping/input/eaton_glcm_ready.tif",
#              "thermal_image_path":"../SAR-Burnt-Area-Mapping/input/eaton_thermal_ready.tif",
#              "dnbr_image_path":"../SAR-Burnt-Area-Mapping/input/eaton_dnbr_ready.tif",
#              }


In [6]:
for image_key in list(images_dict.keys()):
    print(image_key)

    with rasterio.open(images_dict[image_key]) as dataset:
        # Print the properties
        print(f"CRS: {dataset.crs}")
        print(f"Width: {dataset.width}")
        print(f"Height: {dataset.height}")
        print(f"Extent: {dataset.bounds}")
        print(f"Resolution: { dataset.res}")
        print("----------------------------------------")

input_sar_image_path
CRS: EPSG:32611
Width: 1737
Height: 1259
Extent: BoundingBox(left=390284.9643946937, bottom=3778493.0553008374, right=407654.9643946937, top=3791083.0553008374)
Resolution: (10.0, 10.0)
----------------------------------------
glcm_raster_path
CRS: EPSG:32611
Width: 1737
Height: 1259
Extent: BoundingBox(left=390284.9643946937, bottom=3778493.0553008374, right=407654.9643946937, top=3791083.0553008374)
Resolution: (10.0, 10.0)
----------------------------------------
thermal_image_path
CRS: EPSG:32611
Width: 1737
Height: 1259
Extent: BoundingBox(left=390284.9643946937, bottom=3778493.055300838, right=407654.9643946937, top=3791083.055300838)
Resolution: (10.0, 10.0)
----------------------------------------
dnbr_image_path
CRS: EPSG:32611
Width: 1737
Height: 1259
Extent: BoundingBox(left=390284.9643946937, bottom=3778493.055300838, right=407654.9643946937, top=3791083.055300838)
Resolution: (10.0, 10.0)
----------------------------------------


## Clipping data by extent

In [ ]:
import rasterio
from rasterio.windows import from_bounds

# Define the extent to clip (minx, miny, maxx, maxy)
minx, miny, maxx, maxy=390284.9643946937, 3778493.0553008374, 407654.9643946937, 3791083.0553008374
clip_extent = (minx, miny, maxx, maxy)

images_dict_1={
             "eaton_glcm_ready":glcm_raster_path,
             "eaton_thermal_ready":thermal_image_path,
             "eaton_dnbr_ready":dnbr_image_path,
             }

for image_key in list(images_dict_1.keys()):
    # Open the raster dataset
    with rasterio.open(images_dict_1[image_key]) as dataset:
        # Get the window for the specified extent
        window = from_bounds(*clip_extent, dataset.transform)
        # Read the data in that window
        # clipped_data = dataset.read(1, window=window)  # Read the first band in the window
        
        # Get the new transform for the clipped data
        clipped_transform = dataset.window_transform(window)
        clipped_data = dataset.read(1, window=window)
        # Define the metadata for the clipped raster
        clipped_meta = dataset.meta.copy()
        clipped_meta.update({
            'height': clipped_data.shape[0],
            'width': clipped_data.shape[1],
            'transform': clipped_transform
        })
        count=dataset.count

        # bands = dataset.read()
        
        # Write the clipped data to a new file
        with rasterio.open(f'input/{image_key}.tif', 'w', **clipped_meta) as dest:
            for i in range(count):
                clipped_data = dataset.read(i+1, window=window)
                dest.write(clipped_data, i+1)  # Write the first band
